In [1]:
# Importing Packages

import pandas as pd
import numpy as np
import pickle, os
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Concatenate, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import keras.backend as K
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
# Import the dataset
dataset = pd.read_csv('device_data.csv')
dataset['timeslot'] = pd.to_datetime(dataset['timeslot'] , format = "%Y-%m-%d %H:%M:%S")
dataset.ffill(inplace=True)
print(dataset.head())

          hostname                 ifname            timeslot  \
0  nl-xy-switch-01  GigabitEthernet10/0/2 2025-06-19 13:00:00   
1  nl-xy-switch-01  GigabitEthernet10/0/2 2025-06-19 14:00:00   
2  nl-xy-switch-01  GigabitEthernet10/0/2 2025-06-19 15:00:00   
3  nl-xy-switch-01  GigabitEthernet10/0/2 2025-06-19 16:00:00   
4  nl-xy-switch-01  GigabitEthernet10/0/2 2025-06-19 17:00:00   

   max_metric_value  mean_metric_value  
0      5.665394e+07       4.274507e+07  
1      8.265270e+07       5.016041e+07  
2      8.111788e+07       5.324932e+07  
3      6.656552e+07       4.785007e+07  
4      7.372211e+07       4.445678e+07  


In [ ]:
# Compute additional features
def compute_features(dataset):
    df = dataset.copy()
    df["diff"] = abs(df["max_metric_value"] - df["mean_metric_value"])
    df["ratio"] = df["max_metric_value"] / df["mean_metric_value"]
    df["delta_max"] = df["max_metric_value"].diff()
    df["delta_mean"] = df["mean_metric_value"].diff()

    # Z-score calculation with rolling window
    roll_w = 4
    rolling_mean_max = df["max_metric_value"].rolling(window=roll_w, min_periods=2).mean()
    rolling_std_max = df["max_metric_value"].rolling(window=roll_w, min_periods=2).std(ddof=0)
    rolling_std_max = rolling_std_max.replace(0, np.nan)
    df["zscore_max"] = (df["max_metric_value"] - rolling_mean_max) / rolling_std_max

    rolling_mean_mean = df["mean_metric_value"].rolling(window=roll_w, min_periods=2).mean()
    rolling_std_mean = df["mean_metric_value"].rolling(window=roll_w, min_periods=2).std(ddof=0)
    rolling_std_mean = rolling_std_mean.replace(0, np.nan)
    df["zscore_mean"] = (df["mean_metric_value"] - rolling_mean_mean) / rolling_std_mean
    return df

In [ ]:
# Creating a single router name for encoding
dataset["router_name"] = dataset["hostname"] + "-" + dataset["ifname"]
dataset.drop(columns=["hostname", "ifname"], inplace=True)
dataset.head()

,timeslot,max_metric_value,mean_metric_value,router_name
0,2025-06-19 13:00:00,5.665394e+07,4.274507e+07,nl-xy-switch-01-GigabitEthernet10/0/2
1,2025-06-19 14:00:00,8.265270e+07,5.016041e+07,nl-xy-switch-01-GigabitEthernet10/0/2
2,2025-06-19 15:00:00,8.111788e+07,5.324932e+07,nl-xy-switch-01-GigabitEthernet10/0/2
3,2025-06-19 16:00:00,6.656552e+07,4.785007e+07,nl-xy-switch-01-GigabitEthernet10/0/2
4,2025-06-19 17:00:00,7.372211e+07,4.445678e+07,nl-xy-switch-01-GigabitEthernet10/0/2


In [ ]:
# Upsample per interface to 1-minute frequency with linear interpolation
def upsample_per_interface(df):
    upsampled = []
    df['timeslot'] = pd.to_datetime(df['timeslot'])

    for name, group in df.groupby(['router_name']):
        group = group.sort_values('timeslot').copy()
        group = group.set_index('timeslot')

        # Resample to 1-minute frequency and interpolate numeric columns only
        group_resampled = group.resample('1T').interpolate(method='linear')
        
        # Broadcast router_name to all rows
        group_resampled['router'] = name[0]

        group_resampled.drop(columns=['router_name'], inplace=True)

        # Reset index
        upsampled.append(group_resampled.reset_index())

    return pd.concat(upsampled, ignore_index=True)

dataset_upsampled = upsample_per_interface(dataset)

C:\Users\acer\AppData\Local\Temp\ipykernel_12404\2441889244.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  group_resampled = group.resample('1T').interpolate(method='linear')
C:\Users\acer\AppData\Local\Temp\ipykernel_12404\2441889244.py:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  group_resampled = group.resample('1T').interpolate(method='linear')
C:\Users\acer\AppData\Local\Temp\ipykernel_12404\2441889244.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  group_resampled = group.resample('1T').interpolate(method='linear')
C:\Users\acer\AppData\Local\Temp\ipykernel_12404\2441889244.py:10: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before inter

In [ ]:
# Computing features after upsampling
dataset_upsampled.sort_values(['router', 'timeslot'], inplace=True)

final_dataset = dataset_upsampled.groupby('router', group_keys=False).apply(compute_features).reset_index(drop=True)
final_dataset.dropna(subset=['delta_max','delta_mean','ratio','zscore_max'], how='any', inplace=True)
print(final_dataset)

                  timeslot  max_metric_value  mean_metric_value  \
1      2025-06-19 13:01:00      5.708725e+07       4.286866e+07   
2      2025-06-19 13:02:00      5.752056e+07       4.299225e+07   
3      2025-06-19 13:03:00      5.795388e+07       4.311584e+07   
4      2025-06-19 13:04:00      5.838719e+07       4.323943e+07   
5      2025-06-19 13:05:00      5.882050e+07       4.336302e+07   
...                    ...               ...                ...   
478205 2025-07-22 17:56:00      1.085928e+07       9.073793e+06   
478206 2025-07-22 17:57:00      1.089622e+07       9.095917e+06   
478207 2025-07-22 17:58:00      1.093316e+07       9.118041e+06   
478208 2025-07-22 17:59:00      1.097010e+07       9.140164e+06   
478209 2025-07-22 18:00:00      1.100704e+07       9.162288e+06   

                                       router          diff     ratio  \
1       nl-xy-switch-01-GigabitEthernet10/0/2  1.421859e+07  1.331678   
2       nl-xy-switch-01-GigabitEthernet10/0/2  1.

C:\Users\acer\AppData\Local\Temp\ipykernel_12404\1691696292.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_dataset = dataset_upsampled.groupby('router', group_keys=False).apply(compute_features).reset_index(drop=True)


In [ ]:
# Detecting outliers using IQR method
outlier_data = []

cap_columns = ['max_metric_value', 'mean_metric_value', 'delta_max', 'delta_mean', 'diff', 'ratio', 'zscore_max', 'zscore_mean']

def outlier_detection(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    IQR = q3 - q1
    lower, upper = q1 - 1.5 * IQR, q3 + 1.5 * IQR
    return (series < lower) | (series > upper)

# Run outlier detection per router and per column
outlier_summary = []
for router, grp in final_dataset.groupby('router'):
    for col in cap_columns:
        if col not in grp.columns:
            continue
        mask = outlier_detection(grp[col])
        cnt = int(mask.sum())
        outlier_summary.append({'router_name': router, 'column': col, 'outlier_count': cnt})
        print(f"Router={router}  Column={col}  outliers={cnt}")

Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=max_metric_value  outliers=9297
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=mean_metric_value  outliers=9196
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=delta_max  outliers=5760
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=delta_mean  outliers=4260
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=diff  outliers=4354
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=ratio  outliers=2714
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=zscore_max  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=zscore_mean  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=max_metric_value  outliers=1152
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=mean_metric_value  outliers=2129
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=delta_max  outliers=4260
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=delta_mean  outliers=4020
Router=nl-xy-switch-01-GigabitEthernet10/1/3

In [8]:
# Capping continuous values to limit within normal behavior (Based on 99th percentile)

cap_columns = ['max_metric_value', 'mean_metric_value', 'delta_max', 'delta_mean', 'diff', 'ratio', 'zscore_max', 'zscore_mean']

def impute_outliers_series(s: pd.Series) -> pd.Series:
    """Cap values outside IQR fences to the largest/smallest value within the fence (preserve index)."""
    
    if s.dropna().empty:
        return s
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    IQR = q3 - q1
    upper_fence = q3 + 1.5 * IQR
    lower_fence = q1 - 1.5 * IQR

    # choose the max value that is <= upper_fence and min value that is >= lower_fence
    within_upper = s[~(s > upper_fence)]
    within_lower = s[~(s < lower_fence)]
    # fallback to series max/min if no values within fence
    upper = within_upper.max() if not within_upper.empty else s.max()
    lower = within_lower.min() if not within_lower.empty else s.min()

    res = s.copy()
    res.loc[s > upper] = upper
    res.loc[s < lower] = lower
    return res

def impute_outliers_group(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()
    for col in cap_columns:
        if col in g.columns:
            g[col] = impute_outliers_series(g[col])
    return g

final_dataset = final_dataset.groupby('router', group_keys=False).apply(impute_outliers_group).reset_index(drop=True)
print(final_dataset)

                  timeslot  max_metric_value  mean_metric_value  \
0      2025-06-19 13:01:00      5.708725e+07       4.286866e+07   
1      2025-06-19 13:02:00      5.752056e+07       4.299225e+07   
2      2025-06-19 13:03:00      5.795388e+07       4.311584e+07   
3      2025-06-19 13:04:00      5.838719e+07       4.323943e+07   
4      2025-06-19 13:05:00      5.882050e+07       4.336302e+07   
...                    ...               ...                ...   
478195 2025-07-22 17:56:00      1.085928e+07       9.073793e+06   
478196 2025-07-22 17:57:00      1.089622e+07       9.095917e+06   
478197 2025-07-22 17:58:00      1.093316e+07       9.118041e+06   
478198 2025-07-22 17:59:00      1.097010e+07       9.140164e+06   
478199 2025-07-22 18:00:00      1.100704e+07       9.162288e+06   

                                       router          diff     ratio  \
0       nl-xy-switch-01-GigabitEthernet10/0/2  1.421859e+07  1.331678   
1       nl-xy-switch-01-GigabitEthernet10/0/2  1.

C:\Users\acer\AppData\Local\Temp\ipykernel_12404\2489533863.py:35: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_dataset = final_dataset.groupby('router', group_keys=False).apply(impute_outliers_group).reset_index(drop=True)


In [9]:
# Run outlier detection per router and per column
outlier_summary = []
for router, grp in final_dataset.groupby('router'):
    for col in cap_columns:
        if col not in grp.columns:
            continue
        mask = outlier_detection(grp[col])
        cnt = int(mask.sum())
        outlier_summary.append({'router': router, 'column': col, 'outlier_count': cnt})
        print(f"Router={router}  Column={col}  outliers={cnt}")

Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=max_metric_value  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=mean_metric_value  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=delta_max  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=delta_mean  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=diff  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=ratio  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=zscore_max  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/0/2  Column=zscore_mean  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=max_metric_value  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=mean_metric_value  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=delta_max  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=delta_mean  outliers=0
Router=nl-xy-switch-01-GigabitEthernet10/1/3  Column=diff  outliers=0
Rout

In [ ]:
#Encoding the router id
encoder = LabelEncoder()
final_dataset['router_id'] = encoder.fit_transform(final_dataset['router'])
with open(r'd:\AI-ES\Projects\KPN Anamoly case\AIEngineerAssignment\scalers\router_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [ ]:
# Creating sequences per router
routers = final_dataset['router_id'].unique()
router_sequences = {}

for r in routers:
    router_df = final_dataset[final_dataset['router_id'] == r].sort_values('timeslot')
    router_sequences[r] = router_df

In [ ]:
# Build sequences and split per-router (time-based) into train/val/test (80/10/10)
window_size = 6

# prepare lists for train/val/test
X_train, X_val, X_test = [], [], []
y_train, y_val, y_test = [], [], []
router_train, router_val, router_test = [], [], []

numerical_features = ['max_metric_value', 'mean_metric_value', 'diff', 'ratio', 'delta_max', 'delta_mean', 'zscore_max', 'zscore_mean']

for idx, r in enumerate(routers):
    df = router_sequences[r].sort_values('timeslot')
    data = df[numerical_features].values
    target = df[['max_metric_value','mean_metric_value']].values
    
    n = len(data)
    # number of usable samples for sequences
    num_samples = n - window_size
    if num_samples <= 0:
        continue

    # split counts based on samples
    train_count = int(num_samples * 0.8)
    val_count = int(num_samples * 0.1)
    # assigning test count on remaining
    test_count = num_samples - train_count - val_count
    # ensure non-negative
    if test_count < 0:
        test_count = 0

    for s in range(num_samples):
        seq = data[s:s+window_size]
        label = target[s+window_size]
        
        # assign by sample index (time-ordered)
        if s < train_count:
            X_train.append(seq); y_train.append(label); router_train.append(idx)
        elif s < train_count + val_count:
            X_val.append(seq); y_val.append(label); router_val.append(idx)
        else:
            X_test.append(seq); y_test.append(label); router_test.append(idx)

# convert to arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)
router_train = np.array(router_train)
router_val = np.array(router_val)
router_test = np.array(router_test)

print('Built sequences — train samples:', X_train.shape[0], ' val samples:', X_val.shape[0], ' test samples:', X_test.shape[0])

Built sequences — train samples: 382510  val samples: 47810  test samples: 47820


In [ ]:
# Store scaled sequences
X_train_scaled, X_val_scaled, X_test_scaled = [], [], []
Y_train_scaled, Y_val_scaled, Y_test_scaled = [], [], []

# Store router-wise scalers
router_train_scalers = {}
router_test_scalers = {}

for router_idx, r in enumerate(routers):

    # Mask samples belonging to this router
    train_mask = router_train == router_idx
    val_mask = router_val == router_idx
    test_mask = router_test == router_idx

    # Extract router-specific training samples
    X_train_router = X_train[train_mask]  # shape: (samples, window_size, num_features)
    X_val_router = X_val[val_mask]
    X_test_router = X_test[test_mask]

    y_train_router = y_train[train_mask]  # shape: (samples, window_size, num_features)
    y_val_router = y_val[val_mask]
    y_test_router = y_test[test_mask]

    # Reshape to 2D for scaling: (samples * timesteps, num_features)
    X_train_flat = X_train_router.reshape(-1, X_train_router.shape[-1])
    Y_train_flat = y_train[train_mask].reshape(-1, y_train.shape[-1])

    # Fit scaler only on training data for this router
    scaler_train = RobustScaler()
    scaler_train.fit(X_train_flat)

    scaler_test = RobustScaler()
    scaler_test.fit(Y_train_flat)

    # Transform train/val/test
    X_train_scaled_router = scaler_train.transform(X_train_router.reshape(-1, X_train_router.shape[-1])).reshape(X_train_router.shape)
    X_val_scaled_router = scaler_train.transform(X_val_router.reshape(-1, X_val_router.shape[-1])).reshape(X_val_router.shape)
    X_test_scaled_router = scaler_train.transform(X_test_router.reshape(-1, X_test_router.shape[-1])).reshape(X_test_router.shape)

    # Transform targets
    Y_train_scaled_router = scaler_test.transform(y_train_router.reshape(-1, y_train_router.shape[-1])).reshape(y_train_router.shape)
    Y_val_scaled_router = scaler_test.transform(y_val_router.reshape(-1, y_val_router.shape[-1])).reshape(y_val_router.shape)
    Y_test_scaled_router = scaler_test.transform(y_test_router.reshape(-1, y_test_router.shape[-1])).reshape(y_test_router.shape)

    # Append scaled data (in the same order as before)
    X_train_scaled.append(X_train_scaled_router)
    X_val_scaled.append(X_val_scaled_router)
    X_test_scaled.append(X_test_scaled_router)

    # Append scaled data (in the same order as before)
    Y_train_scaled.append(Y_train_scaled_router)
    Y_val_scaled.append(Y_val_scaled_router)
    Y_test_scaled.append(Y_test_scaled_router)

    # Keep track of scalers
    router_train_scalers[r] = scaler_train
    router_test_scalers[r] = scaler_test

# Concatenate all routers back
X_train_scaled = np.concatenate(X_train_scaled, axis=0)
X_val_scaled = np.concatenate(X_val_scaled, axis=0)
X_test_scaled = np.concatenate(X_test_scaled, axis=0)

Y_train_scaled = np.concatenate(Y_train_scaled, axis=0)
Y_val_scaled = np.concatenate(Y_val_scaled, axis=0)
Y_test_scaled = np.concatenate(Y_test_scaled, axis=0)

In [ ]:
#Saving the scalers for both train and test
with open(r'd:\AI-ES\Projects\KPN Anamoly case\AIEngineerAssignment\scalers\router_scalers_train.pkl', 'wb') as f:
    pickle.dump(router_train_scalers, f)

with open(r'd:\AI-ES\Projects\KPN Anamoly case\AIEngineerAssignment\scalers\router_scalers_test.pkl', 'wb') as g:
    pickle.dump(router_test_scalers, g)

In [ ]:
#Define the model architecture with hyperparameters
def lstm(parameters):

    n_routers = len(routers)

    # Input layers
    seq_input = Input(shape=(window_size, len(numerical_features)))
    router_input = Input(shape=(1,))

    # Embedding layer for router
    router_embedding = Embedding(input_dim=n_routers, output_dim=parameters.Int('embeddinglayer', min_value=2, max_value=6, step=2))(router_input)
    router_embedding = Flatten()(router_embedding)

    # LSTM first layer
    x1 = LSTM(parameters.Int('LstmL1', min_value=32, max_value=128, step=32), return_sequences=False)(seq_input)

    # Combining Embedding output with LSTM output
    x_combined = Concatenate()([x1, router_embedding])

    # Output layer
    x = Dense(parameters.Int('DenseL1', min_value=16, max_value=64, step=16), activation=parameters.Choice('dense_activation', values=['relu', 'tanh']), kernel_regularizer=regularizers.l2(1e-4))(x_combined)
    x = Dropout(parameters.Float('dropout', min_value=0.1, max_value=0.3, step=0.1))(x)
    output = Dense(2)(x)

    model = Model(inputs=[seq_input, router_input], outputs=output)
    optimizer = parameters.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])
    learning_rate = parameters.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='mse', metrics =['mae','mse'])

    return model

In [ ]:
#Initializing the tuner
tuner= kt.RandomSearch(lstm,objective='mse',max_trials=20,executions_per_trial=1)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Start the hyperparameter search
tuner.search(
    x=[X_train_scaled, router_train],
    y=Y_train_scaled,
    epochs=10,
    batch_size=32,
    validation_data=([X_val_scaled, router_val], Y_val_scaled),
    callbacks=[early_stopping]
)

Trial 20 Complete [00h 03m 33s]
mse: 0.009754283353686333

Best mse So Far: 0.0023990795016288757
Total elapsed time: 01h 46m 28s


In [ ]:
#Best hyperparameter setting
best_trial = tuner.oracle.get_best_trials(1)[0]
best_hp = best_trial.hyperparameters
print(best_hp.values)

{'embeddinglayer': 4, 'LstmL1': 64, 'DenseL1': 64, 'dense_activation': 'tanh', 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.001}


Best Combination : {'embeddinglayer': 4, 'LstmL1': 64, 'DenseL1': 64, 'dense_activation': 'tanh', 'dropout': 0.1, 'optimizer': 'adam', 'learning_rate': 0.001}

In [21]:
# Build the best model
best_model = tuner.hypermodel.build(best_hp)
best_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 6, 8)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 4)      │         40 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 64)        │     18,688 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 4)         │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 68)        │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      4,416 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 2)         │        130 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,274 (90.91 KB)

 Trainable params: 23,274 (90.91 KB)

 Non-trainable params: 0 (0.00 B)